In [1]:
import sys

import pandas as pd
import numpy as np 

sys.path.append('.\\..\\..\\..\\..')

from src.utils.Utils import save_answer

In [2]:
test_data = pd.read_csv('data/coursera_sessions_test.txt', delimiter=';', header=None)
test_data.columns = ['recommended', 'bought']
print(test_data.shape)
print(test_data.dropna().shape)
test_data.dropna().head()

(50000, 2)
(3665, 2)


,recommended,bought
7,"63,68,69,70,66,61,59,61,66,68","66,63"
14,"158,159,160,159,161,162",162
19,"200,201,202,203,204","201,205"
34,"371,372,371","371,373"
40,422,422


In [3]:
train_data = pd.read_csv('data/coursera_sessions_train.txt', delimiter=';', header=None)
train_data.columns = ['recommended', 'bought']
print(train_data.shape)
print(train_data.dropna().shape)
train_data.dropna().head()

(50000, 2)
(3608, 2)


,recommended,bought
7,"59,60,61,62,60,63,64,65,66,61,67,68,67","67,60,63"
10,"84,85,86,87,88,89,84,90,91,92,93,86",86
19,"138,198,199,127",199
30,"303,304,305,306,307,308,309,310,311,312",303
33,"352,353,352",352


In [4]:
def count(data):
    dictionary = {}
    for s in data:
        if s in dictionary:
            dictionary[s] += 1
        else:
            dictionary[s] = 1
    return dictionary

In [5]:
def uniquifier(seq):
    seen = set()
    return [x for x in seq if not (x in seen or seen.add(x))]

In [6]:
def f(data_to_train, data_to_test):
    # Список товаров для обучения
    veiw_goods = [g for lg in data_to_train for g in lg.split(',')]

    # Удаление пустых сессий
    t_data = data_to_test.dropna()

    # Товары которые были куплены
    bought_goods = [g.split(',') for g in t_data.bought]

    # Справочник с кол-вом повторений
    count_dict = count(veiw_goods)

    # Товары которые были просмотрены пользователем
    view_goods_after_dropna = [g.split(',') for g in t_data.recommended]
    view_goods_after_dropna = [[(g, count_dict[g] if g in count_dict else 0) for g in uniquifier(r)] for r in view_goods_after_dropna]

    # Лист рекомендаций
    recom_list = [[t[0] for t in sorted(l, key=lambda x: x[1], reverse=True)] for l in view_goods_after_dropna]

    # Размер
    size = t_data.shape[0]

    # AverageRecall@1
    ans1_1 = np.sum([(1 if recom_list[idx][0] in bg else 0) / min(len(bg), 5) for idx, bg in enumerate(bought_goods)]) / size
    # AveragePrecision@1
    ans1_2 = np.sum([1 if recom_list[idx][0] in bg else 0 for idx, bg in enumerate(bought_goods)]) / size
    # AverageRecall@5
    ans1_3 = np.sum([np.sum([1 if rl in bg else 0 for rl in recom_list[idx][:5]]) / min(len(bg), 5) for idx, bg in enumerate(bought_goods)]) / size
    # AveragePrecision@5
    ans1_4 = np.sum([np.sum([1 if rl in bg else 0 for rl in recom_list[idx][:5]]) / 5 for idx, bg in enumerate(bought_goods)]) / size
    
    return [round(ans1_1, 2), round(ans1_2, 2), round(ans1_3, 2), round(ans1_4, 2)]
#     return [ans1_1, ans1_2, ans1_3, ans1_4]

In [7]:
save_answer(1, f(train_data.recommended, train_data), space=True)


Task №1 answer [0.44, 0.51000000000000001, 0.82999999999999996, 0.20999999999999999]



In [8]:
save_answer(2, f(train_data.recommended, test_data), space=True)


Task №2 answer [0.41999999999999998, 0.47999999999999998, 0.80000000000000004, 0.20000000000000001]



In [9]:
save_answer(3, f(train_data.dropna().bought, train_data), space=True)


Task №3 answer [0.68999999999999995, 0.80000000000000004, 0.93000000000000005, 0.25]



In [10]:
save_answer(4, f(train_data.dropna().bought, test_data), space=True)


Task №4 answer [0.46000000000000002, 0.53000000000000003, 0.81999999999999995, 0.20999999999999999]

